In [1]:
# syft absolute
import syft as sy
from syft.types.syft_object import SyftObject
from syft import ActionObject
# syft absolute
from syft.service.user.user import UserCreate
from syft.node.credentials import SyftVerifyKey
from syft.service.action.action_permissions import ActionPermission, ActionObjectPermission

from syft.service.job.job_stash import Job

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/pysyft/notebooks


In [2]:
node_low = sy.orchestra.launch(
    name="test_l",
    node_side_type="low",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)
node_high = sy.orchestra.launch(
    name="test_h",
    node_side_type="high",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/8a1c04544655402190588aec30079bc3.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=0f15b3f3404047aca862f7d38da64d39 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/8212e6797fde4c3fba4fc53ab555a886.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=8c7ca1f439d341a692047714e8d4547f in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
client_low = node_low.login(email="info@openmined.org", password="changethis")
client_high = node_high.login(email="info@openmined.org", password="changethis")

Logged into <test_l: Low side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test_h: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [4]:
client_low.register(email="newuser@openmined.org", name="John Doe", password="pw", password_verify="pw")
client_low_ds = node_low.login(email="newuser@openmined.org", password="pw")

Logged into <test_l: Low side Domain> as <newuser@openmined.org>


# create datasets

In [5]:
# third party
import numpy as np

In [6]:
mock_high = np.array([10, 11, 12, 13, 14])
private_high = np.array([15, 16, 17, 18, 19])

dataset_high = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_high,
            data=private_high,
            shape=private_high.shape,
            mock_is_real=True,
        )
    ],
)

client_high.upload_dataset(dataset_high)

  0%|                                                                                                                 | 0/1 [00:00<?, ?it/s]

Uploading: numpy-data


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.69it/s]


SyftSuccess: Dataset uploaded to 'test_h'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [7]:
mock_low = np.array([0, 1, 2, 3, 4])  # do_high.mock
# private_low = np.array([5, 6, 7, 8, 9])  # AOEmpty? create new type AO

dataset_low = sy.Dataset(
    id=dataset_high.id,
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_low,
            data=ActionObject.empty(data_node_id=client_high.id),
            shape=mock_low.shape,
            mock_is_real=True,
        )
    ],
)

res = client_low.upload_dataset(dataset_low)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 13.38it/s]

Uploading: numpy-data


In [8]:
data_low = client_low_ds.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    return data.mean()


compute_mean(data=data_low.mock)

SyftSuccess: Syft function 'compute_mean' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftInfo: Creating a node with n_consumers=2 (the default value)
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/25fba70b891e4ce4b74c0a8d8df3e08b.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=09588fbc6cdf45f18cbeaf057ef0c968 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Logged into <ephemeral_node_compute_mean_3447: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Approving request for domain ephemeral_node_compute_mean_3447
override True
SyftInfo: Landing the ephmeral node...


```python
Pointer
```
2.0

In [9]:
client_low_ds.code.request_code_execution(compute_mean)

syft.service.request.request.Request

In [10]:
request = client_low.requests[0]

In [11]:
code = client_low.code[0]

TEO HERE

In [12]:
from syft.client.syncing import compare_states

low_state = client_low.sync.get_state()
high_state = client_high.sync.get_state()

diff_state = compare_states(low_state, high_state)

In [13]:
diff_state.diffs

compute_mean
test_l
Node: test_l, Status: pending
default-pool


[syft.service.sync.diff_state.Diff, syft.service.sync.diff_state.Diff]

In [14]:
# state.objs_to_sync
from syft.client.syncing import resolve
low_items_to_sync, high_items_to_sync = resolve(diff_state, force_approve=True)

```python
class Request:
  id: str = 9be2f532c13e41a19c4563053c930ba8
  request_time: str = 2024-02-20 16:46:33
  updated_at: str = None
  status: str = RequestStatus.PENDING
  changes: str = ['Request to change <b>compute_mean</b> (Pool Id: <b>default-pool</b>) to permission <b>RequestStatus.APPROVED</b>. No nested requests']
  requesting_user_verify_key: str = d040263d625928b592d220dd773c87ac01980c083ee977b1a9156db5cd8f9f43

```

```python
class UserCode
    id: UID = 8c98d56c15e94ee0a76c5c867dfaa7e4
    service_func_name: str = compute_mean
    shareholders: list = ['test_l']
    status: list = ['Node: test_l, Status: pending']
    
    code:

@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    return data.mean()

```

In [15]:
# request.changes

In [16]:
# items_to_sync = [request, code]

- What do we do with properties here?
  - just set them to None

In [17]:
def get_stash_for_item(item, node):
    services = list(node.python_node.service_path_map.values())

    all_stashes = dict()
    for serv in services:
        if (_stash := getattr(serv, "stash", None)) is not None:
            all_stashes[_stash.object_type] = _stash

    stash = all_stashes.get(type(item), None)
    return stash

In [18]:
def set_obj_ids(x, node_uid, verify_key):
    if hasattr(x, "__dict__") and isinstance(x, SyftObject):
        for prop, val in x.__dict__.items():
            if isinstance(val, (list, tuple)):
                for v in val:
                    set_obj_ids(v, node_uid, verify_key)
            elif isinstance(val, dict):
                for v in val.values():
                    set_obj_ids(v, node_uid, verify_key)
            else:
                set_obj_ids(val, node_uid, verify_key)
        x.syft_node_location=node_uid
        x.syft_client_verify_key=verify_key
        if hasattr(x, "node_uid"):
            x.node_uid=node_uid
                    
                    
    # TODO: hadnle nested collections
            

In [19]:
def add_permissions_for_actionobject(obj, node_to, node_from):
    _id = obj.id.id 
    blob_id = obj.syft_blob_storage_entry_id
    all_permissions = node_from.python_node.get_service("actionservice").store.permissions[_id]
    read_permissions = [x for x in all_permissions if "READ" in x]

    store_to = node_to.python_node.get_service("actionservice").store
    store_to_blob =node_to.python_node.get_service("blobstorageservice").stash.partition
    print("adding permissions")

    for read_permission in read_permissions:
        
        creds, perm_str = read_permission.split("_") 
        perm = ActionPermission[perm_str]
        permission = ActionObjectPermission(uid=_id, permission=perm, credentials=SyftVerifyKey(creds))
        res = store_to.add_permission(permission)
        
        permission_blob = ActionObjectPermission(uid=blob_id, permission=perm, credentials=SyftVerifyKey(creds))        
        res_blob = store_to_blob.add_permission(permission_blob)

        print("Created permission", permission)
        print("Created blob permission", permission, "with blob id", blob_id)
        
    

In [20]:
def create_actionobject(action_object, client, node_to, node_from):
    print("syncing obj with blob id", action_object.syft_blob_storage_entry_id)
    action_object = action_object.refresh_object()
    action_object.send(client)
#     res = client.api.services.action.set(action_object)
    print("Created", res, "blob_id:", action_object.syft_blob_storage_entry_id)

    if node_to.python_node.node_side_type.value == "low":
        add_permissions_for_actionobject(action_object, node_to, node_from)

In [21]:
from syft.client.api import NodeIdentity
from syft.service.code.user_code import UserCode

In [22]:
def transform_item(item, node):
    identity = NodeIdentity.from_node(node.python_node)
    if isinstance(item, UserCode):
        res = {}
        for key, val in item.status.status_dict.items():
            # todo, check if they are actually only two nodes
            res[identity] = code.status.status_dict[key]
        item.status.status_dict=res
    return item

In [23]:
def add_permissions_for_item(item, node_to, node_from):
    _id = item.id 
    all_permissions = node_from.python_node.get_service("jobservice").stash.partition.permissions[_id]
    read_permissions = [x for x in all_permissions if "READ" in x]

    store_to = node_to.python_node.get_service("jobservice").stash.partition
    for read_permission in read_permissions:
        creds, perm_str = read_permission.split("_") 
        perm = ActionPermission[perm_str]
        permission = ActionObjectPermission(uid=_id, permission=perm, credentials=SyftVerifyKey(creds))
        res = store_to.add_permission(permission)
        print("Created permission", res)

In [24]:
def sync_items(items, node_to, node_from, client):
    for item in items:
        
        if isinstance(item, ActionObject):
            create_actionobject(item, client, node_to, node_from)

        else:
                
            item = transform_item(item, node_to)
            stash = get_stash_for_item(item, node_to)
            creds=client.verify_key

            set_obj_ids(item, client.id, creds)
            exists = stash.get_by_uid(creds, item.id).ok() is not None
            if exists:
                res = stash.update(creds, item)
            else:
#                 res = stash.delete_by_uid(node.python_node.verify_key, item.id)
                res = stash.set(creds, item)
            if not res.is_ok():
                raise ValueError("")
            else:
                print("created", item)
                if isinstance(item, Job) and node_to.python_node.node_side_type.value == "low":
                    add_permissions_for_item(item, node_to, node_from)
    client._fetch_api(client.credentials)

In [25]:
len(low_items_to_sync), len(high_items_to_sync)

(0, 2)

In [26]:
sync_items(low_items_to_sync, node_low, node_high, client_low)

In [27]:
sync_items(high_items_to_sync, node_high, node_low, client_high)

SyftInfo: Node Landed!
created syft.service.request.request.Request
created syft.service.code.user_code.UserCode


In [28]:
# sync_items(items_to_sync, node_high, node_low, client_high)

# Run code high and sync back result

In [29]:
data_high = client_high.datasets[0].assets[0]

In [30]:
job_high = client_high.code.compute_mean(data=data_high, blocking=False)
display(job_high)

```python
class Job:
    id: UID = e577ddc5024e4df6b7b924f10e711b93
    status: created
    has_parent: False
    result: syft.service.action.action_data_empty.ObjectNotReady
    logs:

0 
    
```

In [31]:
# wait for the result
job_high.wait().get()

override True


17.0

In [32]:
job_info = job_high.info(public_metadata=True, result=True)

In [33]:
job_info.result

```python
Pointer
```
17.0

In [34]:
request = client_high.requests[0]
result_obj = job_high.result

print("result obj", result_obj.id)
accept_res = request.accept_by_depositing_result(job_info)

# request.approve()
# TODO: FIX THIS
request = client_high.requests[0]
code = request.code
log = job_high._get_log_objs()

result obj fff75f47c06044e68016430e537d1cd5
Approving request for domain test_h
Approving request for domain test_h
ADDING PERMISSION [READ: fff75f47c06044e68016430e537d1cd5 as d040263d625928b592d220dd773c87ac01980c083ee977b1a9156db5cd8f9f43] fff75f47c06044e68016430e537d1cd5
returning existing job
setting permission
None
None


In [35]:
action_store_high = node_high.python_node.get_service("actionservice").store
blob_store_high = node_high.python_node.get_service("blobstorageservice").stash.partition
assert f"{client_low_ds.verify_key}_READ" in action_store_high.permissions[job_high.result.id.id]
assert f"{client_low_ds.verify_key}_READ" in blob_store_high.permissions[job_high.result.syft_blob_storage_entry_id]

## Sync back to low

In [36]:
low_state = client_low.sync.get_state()
high_state = client_high.sync.get_state()

diff_state_2 = compare_states(low_state, high_state)

diff_state_2.diffs


2024-02-20 16:46:35.263362


[syft.service.sync.diff_state.Diff,
 syft.service.sync.diff_state.Diff,
 syft.service.sync.diff_state.Diff,
 syft.service.sync.diff_state.Diff,
 syft.service.sync.diff_state.Diff]

In [50]:
high_state.rows

[syft.service.sync.sync_state.SyncStateItem,
 syft.service.sync.sync_state.SyncStateItem,
 syft.service.sync.sync_state.SyncStateItem,
 syft.service.sync.sync_state.SyncStateItem,
 syft.service.sync.sync_state.SyncStateItem]

In [37]:
low_items_to_sync, high_items_to_sync = resolve(diff_state_2, force_approve=True)

```python
class SyftLog:
  id: str = a3899e7770a14f17afe15821b0eafb2b

```

```python
class Job:
    id: UID = e577ddc5024e4df6b7b924f10e711b93
    status: completed
    has_parent: False
    result: 17.0
    logs:

0 
JOB COMPLETED
    
```

```python
Pointer
```
17.0

In [38]:
low_items_to_sync

error representing <class 'list'> of objects. All arrays must be of the same length, Traceback (most recent call last):
  File "/Users/koen/workspace/PySyft/packages/syft/src/syft/types/syft_object.py", line 773, in list_dict_repr_html
    vals = get_repr_values_table(self, is_homogenous, extra_fields=extra_fields)
  File "/Users/koen/workspace/PySyft/packages/syft/src/syft/types/syft_object.py", line 719, in get_repr_values_table
    df = pd.DataFrame(cols)
  File "/opt/anaconda3/envs/syft/lib/python3.9/site-packages/pandas/core/frame.py", line 664, in __init__
    mgr = dict_to_mgr(data, index, columns, dtype=dtype, copy=copy, typ=manager)
  File "/opt/anaconda3/envs/syft/lib/python3.9/site-packages/pandas/core/internals/construction.py", line 493, in dict_to_mgr
    return arrays_to_mgr(arrays, columns, index, dtype=dtype, typ=typ, consolidate=copy)
  File "/opt/anaconda3/envs/syft/lib/python3.9/site-packages/pandas/core/internals/construction.py", line 118, in arrays_to_mgr
    ind

[syft.service.log.log.SyftLog,
 syft.service.job.job_stash.Job,
 syft.service.request.request.Request,
 syft.service.code.user_code.UserCode,
 Pointer:
 17.0]

In [39]:
sync_items(low_items_to_sync, node_low, node_high, client_low)

created syft.service.log.log.SyftLog
created syft.service.job.job_stash.Job
Created permission None
Created permission None
created syft.service.request.request.Request
created syft.service.code.user_code.UserCode
syncing obj with blob id 54ab4d316a674b2d90a27c12f3ef9a64
Created SyftSuccess: Dataset uploaded to 'test_l'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets` blob_id: 54ab4d316a674b2d90a27c12f3ef9a64
adding permissions
Created permission [READ: fff75f47c06044e68016430e537d1cd5 as d311a667006cbb56614a062b2bbc7a733b5a4a8edd5293e0f34bb5c75f51277d]
Created blob permission [READ: fff75f47c06044e68016430e537d1cd5 as d311a667006cbb56614a062b2bbc7a733b5a4a8edd5293e0f34bb5c75f51277d] with blob id 54ab4d316a674b2d90a27c12f3ef9a64
Created permission [READ: fff75f47c06044e68016430e537d1cd5 as d040263d625928b592d220dd773c87ac01980c083ee977b1a9156db5cd8f9f43]
Created blob permission [READ: fff75f47c06044e68016430e537d1cd5 as d040263d625928b592d220

In [40]:
# items_to_sync = [code, job_high.result, job_high, log]

In [41]:
# sync_items(items_to_sync, node_low, node_high, client_low)

In [42]:
action_store_low = node_low.python_node.get_service("actionservice").store
blob_store_low = node_low.python_node.get_service("blobstorageservice").stash.partition
assert f"{client_low_ds.verify_key}_READ" in action_store_low.permissions[job_high.result.id.id]
assert f"{client_low_ds.verify_key}_READ" in blob_store_low.permissions[job_high.result.syft_blob_storage_entry_id]

# Run code low

## Run

In [43]:
res_low = client_low_ds.code.compute_mean(data=data_low)

override False


In [44]:
res_low.get()

17.0

In [45]:
assert res_low.get() == private_high.mean()
assert res_low.id == job_high.result.id.id == code.output_policy.last_output_ids[0].id.id
assert job_high.result.syft_blob_storage_entry_id == res_low.syft_blob_storage_entry_id

In [46]:
private_high.mean()

17.0

In [47]:
job_low = client_low_ds.code.compute_mean(data=data_low, blocking=False)

8c98d56c15e94ee0a76c5c867dfaa7e4


SyftWarning: There are existing jobs for this user code, returning the latest one

In [48]:
job_low.wait().get()

17.0

In [49]:
assert job_low.id == job_high.id
assert job_low.result.id == job_high.result.id
assert job_low.result.syft_blob_storage_entry_id == job_high.result.syft_blob_storage_entry_id